In [1]:
import os
os.environ["SSL_CERT_FILE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"
os.environ["REQUESTS_CA_BUNDLE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/processed_data/all_cities_combined_clean.csv")

In [4]:
df

,id,city,section,subsection,text
0,0,Cairo,See,Further afield,29.8 31.233333 1 Dahshur Pyramids . For a cont...
1,1,Cairo,See,NaN,29.8 31.233333 5 Dahshur Pyramids . For a cont...
2,2,Cairo,See,NaN,30.0405 31.2646 2 Al-Azhar Park . A landscaped...
3,3,Cairo,See,NaN,30.0417 31.2483 4 Abdeen Palace . About 1 km e...
4,4,Cairo,See,NaN,30.0475 31.262222 3 Khan El Khalily . Cairo's ...
...,...,...,...,...,...
539,539,Seoul,Districts,NaN,"West ( Gangseo-gu, Yangcheon-gu ) Western area..."
540,540,Seoul,See,NaN,While Seoul today is mostly known as a super-m...
541,541,Seoul,Districts,NaN,Yeongdeungpo & Guro (영등포 / 구로) ( Yeongdeungpo-...
542,542,Seoul,Districts,NaN,Yongsan (용산) ( Yongsan-gu ) Yongsan is home to...


In [5]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py --no-check-certificate

--2025-09-23 13:11:03--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4273 (4.2K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   4.17K  --.-KB/s    in 0.002s  

2025-09-23 13:11:04 (1.75 MB/s) - ‘minsearch.py.1’ saved [4273/4273]



In [6]:
import minsearch

/mnt/d/Travel Assistant/Musafir/notebooks/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [7]:
df.columns

Index(['id', 'city', 'section', 'subsection', 'text'], dtype='object')

In [8]:
documents = df.to_dict(orient='records')

In [9]:
documents[15]

{'id': 15,
 'city': 'Cairo',
 'section': 'Eat',
 'subsection': 'Self-catering',
 'text': 'By far the cheapest and most satisfying option, buying from souks and outdoor markets makes for a crash course in Arabic and haggling, not to mention that the produce is often superb! Bread can be found on nearly every corner and comes in two types - whole wheat aysh baladi and white flour aysh shami . Both are baked fresh daily and delivered by thousands of kids on bicycles to every corner of the city. Every neighborhood has a few streets dedicated to produce and other goods. Always wash fruit thoroughly before eating. Eating a fresh Roma tomato in the heat of Summer straight from a market seller after being washed is a delight, hard to match. The fruits and vegetables in Egypt may not conform to EU or US standards of size, but their taste is far superior.'}

In [10]:
index = minsearch.Index(
    text_fields=['city', 'section', 'text'],
    keyword_fields=['id']
)

In [11]:
index.fit(documents)

In [12]:
prompt_template = """
You emulate a user of our travel assistant application.
Formulate 5 questions this user might ask based on a provided text.
Make the questions specific to this text.
The record should contain the answer to the questions, and the questions should be 
complete and not too short. Use as fewer words as possible from the record. 

The record: 
City: {city},
Section: {section},
Subsection: {subsection},
Text: {text}

Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [18]:
prompt = prompt_template.format(**documents[0])

In [19]:
print(prompt)

You emulate a user of our travel assistant application.
Formulate 5 questions this user might ask based on a provided text.
Make the questions specific to this text.
The record should contain the answer to the questions, and the questions should be 
complete and not too short. Use as fewer words as possible from the record. 

The record: 
City: Cairo,
Section: See,
Subsection: Further afield,
Text: 29.8 31.233333 1 Dahshur Pyramids . For a contrast to touristy Pyramids of Giza, head south to the oldest known pyramid, the Red Pyramid. The neglected Dahshur Pyramids are interesting and worth a visit, considering its history and the hassle-free atmosphere. Also, see the weird Bent Pyramid there and hike around the area to the Black Pyramid. The Red Pyramid has an entrance to the inside, which you can climb down. ( updated Jan 2018 )

Provide the output in parsable JSON without using code blocks:

{"questions": ["question1", "question2", ..., "question5"]}


# RAG Flow


In [20]:
from mistralai import Mistral
from mistralai.models import UserMessage
import os
from dotenv import load_dotenv

In [21]:
# loads variables from .env
load_dotenv()  

True

In [22]:
api_key = os.getenv("API_KEY")

In [24]:
client = Mistral(api_key = api_key)
response = client.chat.complete(
    model= "ministral-8b-latest",
    messages=[UserMessage(content=prompt)],
)

print(response.choices[0].message.content)

{"questions": [
  "What are the coordinates of the Dahshur Pyramids?",
  "Which pyramid is the oldest known pyramid at Dahshur?",
  "Can I enter the Red Pyramid and climb down inside?",
  "What is the Bent Pyramid like?",
  "Is there a Black Pyramid at Dahshur?"
]}


# Generate Questions


In [26]:
def generate_question(doc):
    prompt = prompt_template.format(**doc)
    client = Mistral(api_key = api_key)
    response = client.chat.complete(
        model= 'mistral-large-2411',
        messages=[UserMessage(content=prompt)],
    )    
    json_response = response.choices[0].message.content
    return json_response

In [30]:
from tqdm.auto import tqdm
import json

In [31]:
results = {}

In [60]:
for doc in tqdm(documents):
    doc_id = doc['id']
    if doc_id in results:
        continue 
    questions_raw = generate_question(doc)
    questions = json.loads(questions_raw)
    results[doc_id] = questions['questions']

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 544/544 [00:47<00:00, 11.51it/s]


In [61]:
len(results)

544

In [62]:
# save the results

final_results = []
for doc_id, qs in results.items():
    for q in qs:
        final_results.append((doc_id, q))

In [63]:
final_results[10]

(2, 'What is the name of the landscaped garden in Cairo?')

In [64]:
df_results = pd.DataFrame(final_results, columns=['id', 'question'])

In [65]:
df_results.to_csv('groud-truth-retrieval.csv', index=False)

In [66]:
!head groud-truth-retrieval.csv

id,question
0,Can I enter the Red Pyramid?
0,What is the main advantage of visiting Dahshur Pyramids over the Pyramids of Giza?
0,How many pyramids can be visited in the Dahshur area?
0,Is it possible to walk around the area to see the Black Pyramid?
0,Which pyramid is noted for its peculiar shape in Dahshur?
1,What's an alternative site to visit instead of the Giza Pyramids?
1,Which is the oldest known pyramid mentioned?
1,What makes the Dahshur Pyramids worth visiting?
1,What activities can be done around the Bent Pyramid?


# Retrieval Evaluation


In [67]:
df_question = pd.read_csv('groud-truth-retrieval.csv')

In [68]:
df_question

,id,question
0,0,Can I enter the Red Pyramid?
1,0,What is the main advantage of visiting Dahshur...
2,0,How many pyramids can be visited in the Dahshu...
3,0,Is it possible to walk around the area to see ...
4,0,Which pyramid is noted for its peculiar shape ...
...,...,...
2715,543,What is the estimated cost of meals in Seoul?
2716,543,How much should I budget for eating out in Seoul?
2717,543,What is the price range for dining in Seoul?
2718,543,Can you tell me the typical cost of a meal in ...


In [69]:
ground_truth = df_question.to_dict(orient='records')

In [70]:
ground_truth[0]

{'id': 0, 'question': 'Can I enter the Red Pyramid?'}

In [71]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [72]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [73]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [74]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2720/2720 [00:19<00:00, 137.79it/s]


{'hit_rate': 0.8636029411764706, 'mrr': 0.7254728349673186}

# Finding the best paramters


In [75]:
df_validation = df_question[:100]
df_test = df_question[100:]

In [76]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [77]:
gt_val = df_validation.to_dict(orient='records')

In [78]:
def minsearch_search(query, boost=None):
    
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [79]:
df.columns

Index(['id', 'city', 'section', 'subsection', 'text'], dtype='object')

In [80]:
param_ranges = {
 'city': (0.0, 3.0),
 'section': (0.0, 3.0),
 'subsection': (0.0, 3.0),
 'text': (0.0, 3.0),
}

In [81]:
def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [82]:
simple_optimize(param_ranges, objective, n_iterations=20)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 123.05it/s]


({'city': 2.997859127240021,
  'section': 0.43974423418358655,
  'subsection': 0.0007198525610273565,
  'text': 2.6179612958375116},
 0.7893571428571429)

In [85]:
# Apply the boost params in the whole dataset
def minsearh_improved(query):
    
    boost = {
          'city': 2.99,
          'section': 0.43,
          'subsection': 0.00,
          'text': 2.61,
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results


evaluate(ground_truth, lambda q: minsearh_improved(q['question']))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2720/2720 [00:24<00:00, 113.19it/s]


{'hit_rate': 0.8669117647058824, 'mrr': 0.7287146942110163}

{'hit_rate': 0.8636029411764706, 'mrr': 0.7254728349673186}